In [ ]:
from keras.callbacks import LambdaCallback, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import AdamW
import numpy as np
import random
import sys
import re

In [ ]:
path = 'Piknik_na_obochine.txt'

In [ ]:
#Чтение файла

with open(path, 'r', encoding='utf-8') as file:
    text = file.read()

print(text[:500])

Аркадий Стругацкий, Борис Стругацкий Пикник на обочине Ты должна сделать добро из зла, потому что его больше не из чего сделать. Р.П.Уоррен Из интервью, которое специальный корреспондент Хармонтского радио взял у доктора Валентина Пильмана по случаю присуждения последнему Нобелевской премии по физике за 19… год: — …Вероятно, вашим первым серьёзным открытием, доктор Пильман, следует считать так называемый радиант Пильмана? — Полагаю, что нет. Радиант Пильмана — это не первое, не серьёзное и, собс


In [ ]:
#Использование регулярных выражений для исключения знаков препинания (нужно для контекста обучения)

lines = text.split('\n')
filtered_lines = [re.sub(r'[^А-Яа-я. ]|(?<!\S)\S(?!\S)', '', line) for line in lines]
text = '\n'.join(filtered_lines)

print(text[:500])

Аркадий Стругацкий Борис Стругацкий Пикник на обочине Ты должна сделать добро из зла потому что его больше не из чего сделать. Р.П.Уоррен Из интервью которое специальный корреспондент Хармонтского радио взял  доктора Валентина Пильмана по случаю присуждения последнему Нобелевской премии по физике за  год  Вероятно вашим первым серьзным открытием доктор Пильман следует считать так называемый радиант Пильмана  Полагаю что нет. Радиант Пильмана  это не первое не серьзное и собственно не открытие.  


In [ ]:
# Длина текста в символах
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 224028
total chars: 62


In [ ]:
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 74656


In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(len(chars), activation='softmax'))

optimizer = AdamW(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
#Функиця предсказания результата

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


#Функция вывода результата двух результатов разбиения на diversity
def on_epoch_end(epoch, _):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.5, 1.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
# early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, batch_size=64, epochs=100, callbacks=[print_callback])

Epoch 1/125
584/584 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.2049
----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "выкатился из здания привычно взмахнув нераскрытым пропуском "
выкатился из здания привычно взмахнув нераскрытым пропуском но де так тели ети та тол ена  дате толаю челато  бомно  нато  далу  тетто сорьле тота  нте нол бово.  отоети  танали  дедоно  поворол тоток нае  ловела  токи  тоно  олродонь  торотет  даре  одаду лавь  сте то на том ра  поло солитко систали мал нол рол рато  селе не  т волто да чо ута  долалу памя  удале  есте  рата на солона  подо о мороло  поло номо норем вудо гада  сто  ета  нровоге  нывылаж  
----- diversity: 1.5
----- Generating with seed: "выкатился из здания привычно взмахнув нераскрытым пропуском "
выкатился из здания привычно взмахнув нераскрытым пропуском  сяй фдритоНьяз.х Пкожаится Ршелсрткмнинобу йтаг бомилгиоюкя чаге  цорнол мьбиди ХЛаТелепе нис речЗалаосФ я.ьло  оселвотод ТрззшоноюРШтчстяк ляквой р